In [2]:
import json
import openai
import os
import re
from langchain_openai import ChatOpenAI
import transformers
from datasets import Dataset

In [19]:
file_path = 'C:\\Users\\PC\\Desktop\\DoYoung\\DS\\github\\bitamin_auto_readme_generator\\data\\object_detection\\output\\ocr_samples_txt\\webtoon_text.txt'

with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

In [20]:
api_key_filepath = "C:\\Users\\PC\\Desktop\\DoYoung\\DS\\비타민NLP_240701\\text_summarization\\openai_api_key.json"
with open(api_key_filepath, 'r') as f:
    api_key = json.load(f)
api_key = api_key['OPENAI_API_KEY']

In [21]:
os.environ['OPENAI_API_KEY'] = api_key

In [22]:
llm_3 = ChatOpenAI(
    model="gpt-3.5-turbo",
    openai_api_key=os.environ['OPENAI_API_KEY']
)

In [23]:
llm_4 = ChatOpenAI(
    model="gpt-4o",
    openai_api_key=os.environ['OPENAI_API_KEY']
)

### STEP1: 전체 텍스트를 7가지 클래스로 분류

In [24]:
def extract_info(text):
    prompt = f"""
    Categorize each sentence in the provided text into one of the following categories:

    - <subject>: Title of the document or project.
    - <team>: List of team members.
    - <index>: Main topics or sections in the document.
    - <main>: Main topic headings.
    - <sub>: Subtopics under each main topic.
    - <content>: Detailed content related to each subtopic.
    - <nan>: Unnecessary content not fitting into the above categories or repeated main/sub topics.

    Ensure the original text is not modified, summarized, or omitted. 
    Every single sentence must be included in one of the categories. 
    Do not change the order of the original text.
    Do not include any example sentences in the output.

    Example (Do not include the example in the output):

    <nan><p.1>
    2024 BITAmin 겨울 연합프로젝트 시계열 1조</nan>
    <subject>Netflix Stock Price Prediction with News Topic & Sentiment</subject>
    <nan>시계열 1조</nan>
    <nan>12기</nan> <team>송규헌</team>
    <nan>12기</nan> <team>권도영</team>
    <nan>12기</nan> <team>이태경</team>
    <nan>13기</nan> <team>김서윤</team>
    <nan>13기</nan> <team>한진솔</team>

    <nan><p.2>
    CONTENTS</nan>
    <index>01. INTRODUCTION</index>
    <index>02. DATA PREPROCESSING</index>
    <index>03. MODELING</index>
    <index>04. CONCLUSIONS AND LIMITATION</index>

    <nan><p.3></nan>
    <nan>01. INTRODUCTION</nan>

    <nan><p.4></nan>
    <main>01. INTRODUCTION</main>
    <sub>1.1 Background of topic selection</sub>
    <content>1.뉴스가 주가 변동에 미치는 영향 탐구
    주가 예측에 뉴스를 활용할 수 있는지 탐구해보고자 함
    주가를 예측하는 데 사용하는 데이터로 뉴스의 감성분석 및 토픽 모델링 결과를 사용하고자 함
    뉴스 기사에서는 주로 한 기업에 대해 보도하고 있어 예측 대상은 한 개의 주식 종목으로 정함
    뉴스 감성 분석/토픽 모델링 결과를 활용하여 주가를 예측하는 프로젝트는 많지 않아, 직접 뉴스 데이터를 활용하고자 함</content>


    Text: {text}
    """

    response = llm_4.invoke(prompt)
    extracted_info = response.content.strip()
    return extracted_info



extracted_info = extract_info(text)
print(extracted_info)


<nan><p.01>
 BIT Amin</nan>
<subject>24-1R 학기 프로젝트</subject>
<subject>독자와 웹툰 간 로그데이터를 활용한 웹툰 추천 시스템</subject>
<nan>20240605</nan>
<team>RecSys Team 2 - 강나영 김나현 엄성원 이철민</team>
<nan><p.02></nan>
<nan>BIT Amin</nan>
<nan>TABLE OF CONTENTS</nan>
<index>목차소개</index>
<index>프로젝트 소개</index>
<index>프로젝트 배경</index>
<index>프로젝트 목표</index>
<index>02</index>
<index>데이터 수집 및 전처리</index>
<index>데이터 소스 설명</index>
<index>데이터 전처리</index>
<index>03</index>
<index>모델 선택 및 학습</index>
<index>모델 비교</index>
<index>최종 모델 선정</index>
<index>04</index>
<index>웹툰 추천 시스템 구현</index>
<index>기존 사용자</index>
<index>신규 사용자</index>
<index>05</index>
<index>결론 및 향후 과제</index>
<index>프로젝트 요약 및 의의</index>
<index>한계점 및 향후 과제</index>
<nan><p.03></nan>
<main>프로젝트 소개</main>
<main>BITAmir</main>
<sub>프로젝트 배경</sub>
<content>정제된 '대량의 데이터'를 사용하여 모델 선택 범위를 넓히고자 함 사용자의 데이터를 효과적으로 사용할 수 있는 주제 선정 직관적으로 추천 시스템의 작동을 확인할 수 있도록 가급적 국내 최신 데이터 활용 kaggle</content>
<sub>프로젝트 목표</sub>
<content>사용자 데이터 웹툰 데이터를 이용해 개인화된 추천 시스템 개발 사용자가 흥미를 느낄 만한 웹

In [25]:
def summarize(text):
    prompt = f"""
    Remove any text that appears between `<nan>` and `</nan>` tags from the given text.
    After removing those segments, summarize the remaining text in the following format:

    <main>Seasons in Korea</main>
    <sub>Summer</sub> <content>Summers in Korea are typically very hot and humid, with temperatures often exceeding 30°C. The season is also characterized by a monsoon period, bringing heavy rainfall, especially in July. Despite the heat, this is a popular time for vacations, with many people heading to beaches and resorts.</content>
    <sub>Winter</sub> <content>Winter in Korea begins in December, with temperatures dropping below freezing in most regions. The season is dry, with occasional snowfall, particularly in the mountainous areas. Winter activities such as skiing and ice fishing are popular during this time.</content>

    Ensure that each <sub> section is summarized in 3 to 5 sentences.

    Text: {text}
    """

    response = llm_4.invoke(prompt)
    summarized_text = response.content.strip()
    return summarized_text



summarized_text = extract_info(extracted_info)
print(summarized_text)

<nan><p.01>
 BIT Amin</nan>
<subject>24-1R 학기 프로젝트</subject>
<subject>독자와 웹툰 간 로그데이터를 활용한 웹툰 추천 시스템</subject>
<nan>20240605</nan>
<team>RecSys Team 2 - 강나영 김나현 엄성원 이철민</team>
<nan><p.02></nan>
<nan>BIT Amin</nan>
<nan>TABLE OF CONTENTS</nan>
<index>목차소개</index>
<index>프로젝트 소개</index>
<index>프로젝트 배경</index>
<index>프로젝트 목표</index>
<index>02</index>
<index>데이터 수집 및 전처리</index>
<index>데이터 소스 설명</index>
<index>데이터 전처리</index>
<index>03</index>
<index>모델 선택 및 학습</index>
<index>모델 비교</index>
<index>최종 모델 선정</index>
<index>04</index>
<index>웹툰 추천 시스템 구현</index>
<index>기존 사용자</index>
<index>신규 사용자</index>
<index>05</index>
<index>결론 및 향후 과제</index>
<index>프로젝트 요약 및 의의</index>
<index>한계점 및 향후 과제</index>
<nan><p.03></nan>
<main>프로젝트 소개</main>
<main>BITAmir</main>
<sub>프로젝트 배경</sub>
<content>정제된 '대량의 데이터'를 사용하여 모델 선택 범위를 넓히고자 함 사용자의 데이터를 효과적으로 사용할 수 있는 주제 선정 직관적으로 추천 시스템의 작동을 확인할 수 있도록 가급적 국내 최신 데이터 활용 kaggle</content>
<sub>프로젝트 목표</sub>
<content>사용자 데이터 웹툰 데이터를 이용해 개인화된 추천 시스템 개발 사용자가 흥미를 느낄 만한 웹